<h1 align="center">Fine-tune BERT Model for Named Entity Recognition in Google Colab</h1>

Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro

A seguir instalamos algumas bibliotecas necessárias para trabalhar com `Transformer Hugging Face`:

```
!pip install datasets -q
!pip install tokenizers -q
!pip install transformers -q
!pip install seqeval -q
```

* Biblioteca `datasets` para buscar dados

* `tokenizers` para pré-processar os dados

* `transformers` para ajustar (fine-tune) os modelos

* `seqeval` para calcular as métricas do modelo

# Dataset

Usaremos um conjunto de dados `NER em inglês` do módulo de conjuntos de dados HuggingFace .

Há um total de $4$ classes, Pessoa(`PER`), Organização(`ORG`), Localização(`LOC`) e outras(`O`).

In [1]:
from datasets import load_dataset

dataset = load_dataset('wikiann', 'en')

/home/tesla/Eddy_Giusepe_Deep_Learning/NLP_Transformers/venv_transformers/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset wikiann (/home/tesla/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)
100%|██████████| 3/3 [00:00<00:00, 662.05it/s]


In [2]:
label_names = dataset["train"].features["ner_tags"].feature.names

label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

O conjunto de treinamento tem $20000$ amostras e o conjunto de validação e teste tem $10000$ amostras cada.

In [3]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 20000
    })
})

In [4]:
dataset['train']

Dataset({
    features: ['tokens', 'ner_tags', 'langs', 'spans'],
    num_rows: 20000
})

# Processamento de Dados

O conjunto de dados tem a seguinte aparência, tem quatro chaves – `'tokens'`, `'ner_tags'`, `'langs'`, `'spans'`. 

In [5]:
dataset['train'][:3]

{'tokens': [['R.H.',
   'Saunders',
   '(',
   'St.',
   'Lawrence',
   'River',
   ')',
   '(',
   '968',
   'MW',
   ')'],
  [';', "'", "''", 'Anders', 'Lindström', "''", "'"],
  ['Karl', 'Ove', 'Knausgård', '(', 'born', '1968', ')']],
 'ner_tags': [[3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 2, 0, 0],
  [1, 2, 2, 0, 0, 0, 0]],
 'langs': [['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en'],
  ['en', 'en', 'en', 'en', 'en', 'en', 'en'],
  ['en', 'en', 'en', 'en', 'en', 'en', 'en']],
 'spans': [['ORG: R.H. Saunders', 'ORG: St. Lawrence River'],
  ['PER: Anders Lindström'],
  ['PER: Karl Ove Knausgård']]}

Os dados precisam ser processados ​​em um formato exigido pelo modelo dos `Transformers`.

* `Bert` espera entradas nos formatos `input_ids`, `token_type_ids` e `attention_mask`

* O rótulo (`label`) também requer ajuste devido à tokenização de subpalavra usada pelo `BERT`

Processaremos os tokens usando o tokenizador de modelo pré-treinado `distilbert-base-uncased`.

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Vamos ver por que precisamos `ajustar os rótulos` (labels) de acordo com a saída de `tokenização`.

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["tokens"], padding="max_length", truncation=True, is_split_into_words=True)
tokenized_datasets_ = dataset.map(tokenize_function, batched=True)


Loading cached processed dataset at /home/tesla/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e/cache-609d873257891e9f.arrow
100%|██████████| 10/10 [00:01<00:00,  5.81ba/s]
Loading cached processed dataset at /home/tesla/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e/cache-591af6fd0a297aac.arrow


* `padding` é definido como `max_length`, para preencher a sequência com o comprimento máximo no conjunto de dados

* `truncation` é definido como `True`, para truncar qualquer sequência que tenha um comprimento maior que o comprimento máximo do modelo aceito (`512` para bert)

* `is_split_into_words` é definido como `True`, pois o conjunto de dados contém os tokens em vez de texto


Se verificarmos o comprimento de `input_ids` e `ner_tags` de `tokenized_datasets_`, não corresponderá:

In [8]:
len(tokenized_datasets_['train'][0]['input_ids'])  == len(tokenized_datasets_['train'][0]['ner_tags'])


False

<font color="red">Esta é a razão pela qual exigimos ajustar os rótulos de acordo com a saída tokenizada.</font>

In [9]:
# Link de estudo --> https://www.freecodecamp.org/news/getting-started-with-ner-models-using-huggingface/
# Obtenha os valores para input_ids, Attention_mask e rótulos ajustados
def tokenize_adjust_labels(samples):
    tokenized_samples = tokenizer.batch_encode_plus(samples["tokens"],
                                                    is_split_into_words=True,
                                                    truncation=True)

    total_adjusted_labels = []
    print(len(tokenized_samples["input_ids"]))
    for k in range(0, len(tokenized_samples["input_ids"])):
        prev_wid = -1
        word_ids_list = tokenized_samples.word_ids(batch_index=k)
        existing_label_ids = samples["ner_tags"][k]
        i = -1
        adjusted_label_ids = []
        for word_idx in word_ids_list:
            # Os tokens especiais têm um ID de palavra que é None. Definimos o rótulo como -100 
            # para que sejam automaticamente ignorados na função de Loss.
            if(word_idx is None):
                adjusted_label_ids.append(-100)
        
            elif(word_idx!=prev_wid):
                i = i + 1
                adjusted_label_ids.append(existing_label_ids[i])
                prev_wid = word_idx

            else:
                label_name = label_names[existing_label_ids[i]]
                adjusted_label_ids.append(existing_label_ids[i])


        total_adjusted_labels.append(adjusted_label_ids)
    # Adicionar labels ajustados às amostras tokenizadas
    tokenized_samples["labels"] = total_adjusted_labels

    return tokenized_samples


A função `tokenize_adjust_labels` pega amostras (objeto de conjunto de dados bruto) como entrada e tokeniza usando o objeto tokenizador. Para entender os `word_ids`, considere o seguinte exemplo:


In [10]:
out = tokenizer("Fine tune NER in google colab!")
out

{'input_ids': [101, 2986, 8694, 11265, 2099, 1999, 8224, 15270, 2497, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
out.word_ids(0)

[None, 0, 1, 2, 2, 3, 4, 5, 5, 6, None]

Aqui, podemos ver que os `IDs` $2$ e $5$ são repetidos duas vezes devido à `tokenização da subpalavra`, então repetiremos o rótulo para as subpalavras (`sub-words`).

Vamos definir o `label` como `-100` para tokens especiais, pois `word id` é `None` para eles e isso será automaticamente ignorado na função de perda (`Loss`).

Aplicaremos `tokenize_adjust_labels` a todo o conjunto de dados usando a função `map`.

In [12]:
tokenized_dataset = dataset.map(tokenize_adjust_labels,
                                batched=True,
                                remove_columns=['tokens', 'ner_tags', 'langs', 'spans'])
                                

Loading cached processed dataset at /home/tesla/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e/cache-600d88128304f51f.arrow
 50%|█████     | 5/10 [00:00<00:00, 19.65ba/s]

1000
1000
1000
1000
1000


 90%|█████████ | 9/10 [00:00<00:00, 17.65ba/s]

1000
1000
1000
1000


100%|██████████| 10/10 [00:00<00:00, 18.10ba/s]
Loading cached processed dataset at /home/tesla/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e/cache-14a13a46c284ef6c.arrow


1000


Vamos remover as colunas que não são necessárias usando o parâmetro `remove_columns`.

Abaixo está o que nosso ` tokenized_dataset ` contém:

In [13]:
tokenized_dataset

DatasetDict({
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
})

Agora, vamos ver algumas amostras do `tokenized_dataset`:

In [14]:
tokenized_dataset['train'][:3]

{'input_ids': [[101,
   1054,
   1012,
   1044,
   1012,
   15247,
   1006,
   2358,
   1012,
   5623,
   2314,
   1007,
   1006,
   5986,
   2620,
   12464,
   1007,
   102],
  [101,
   1025,
   1005,
   1005,
   1005,
   15387,
   11409,
   5104,
   13887,
   1005,
   1005,
   1005,
   102],
  [101, 6382, 1051, 3726, 14161, 20559, 13444, 1006, 2141, 3380, 1007, 102]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'labels': [[-100, 3, 3, 3, 3, 4, 0, 3, 3, 4, 4, 0, 0, 0, 0, 0, 0, -100],
  [-100, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, -100],
  [-100, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, -100]]}

Como podemos ver, amostras diferentes têm comprimentos diferentes, portanto, precisamos preencher os tokens para que tenham o mesmo comprimento. Para isso, usaremos o [DataCollatorForTokenClassification](https://huggingface.co/docs/transformers/main/main_classes/data_collator#transformers.DataCollatorForTokenClassification), ele preencherá tanto as entradas quanto os rótulos.


In [15]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [16]:
data_collator

DataCollatorForTokenClassification(tokenizer=PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), padding=True, max_length=None, pad_to_multiple_of=None, label_pad_token_id=-100, return_tensors='pt')

## <font color="red">Fine-tune</font>

* Usaremos o modelo `Distillbert-base-uncased` para ajuste fino (`fine-tuning`)

* Precisamos especificar o número de rótulos presentes no dataset

In [17]:
from transformers import AutoModelForTokenClassification
#from transformers import AutoTokenizer

model = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased",
                                                        num_labels=len(label_names))


Downloading: 100%|██████████| 268M/268M [00:24<00:00, 10.8MB/s] 
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['c

Precisamos definir uma função que possa processar nossas previsões de modelo e calcular as métricas necessárias. Usaremos `métricas` [seqeval](https://github.com/chakki-works/seqeval), comumente usadas para `classificação de token`.

In [18]:
import numpy as np
from datasets import load_metric

metric = load_metric("seqeval")

/tmp/ipykernel_62227/2794963309.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [19]:
metric

Metric(name: "seqeval", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of shape (n_samples,), weights for indi

In [20]:
def compute_metrics(p):
    predictions, labels = p
    # Select predicted index with maximum logit for each token
    predictions = np.argmax(predictions, axis=2)
    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

A entrada (`input`) para a função `compute_metrics` é nomeada tupla contendo previsões do modelo e rótulos correspondentes. Vamos pegar o `logit máximo` para cada token e ignorar os tokens especiais (`-100`) como um rótulo que definimos na parte de pré-processamento. Por fim, retornaremos o dicionário que consiste nas métricas `Precision`, `Recall`, `F1-score` e `Accuracy`.


Como nossa função de dataset e métricas está pronta, podemos ajustar (`fine-tune`) o modelo usando `Trainer API`.

In [22]:
from transformers import TrainingArguments, Trainer

batch_size = 16
logging_steps = len(tokenized_dataset['train']) // batch_size
epochs = 2

Executaremos o treinamento por $2$ `epochs` com `batch_size=16`, depois podemos brincar com esses parâmetros para melhorar o desempenho do nosso modelo, se necessário.

Para usar o `Trainer API`, precisamos definir argumentos de treinamento que contenham atributos para customizar o treinamento. Para entender todos os parâmetros que ele suporta, consulte a [documentação](https://huggingface.co/docs/transformers/v4.19.2/en/main_classes/trainer#transformers.TrainingArguments).

In [23]:
training_args = TrainingArguments(output_dir="results",
                                  num_train_epochs=epochs,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps) 

Agora, vamos instanciar o objeto `Trainer` e passar o modelo, datasets, argumentos de treinamento, tokenizador, função `compute_metrics` para calcular métricas e o `data _collator` para preenchimento.

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [27]:
#fine tune using train method
trainer.train()

***** Running training *****
  Num examples = 20000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2500
  Number of trainable parameters = 66368263
 20%|██        | 500/2500 [00:22<01:20, 24.87it/s]Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json
Model weights saved in results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-500/tokenizer_config.json
Special tokens file saved in results/checkpoint-500/special_tokens_map.json
 40%|███▉      | 998/2500 [00:49<01:06, 22.64it/s]Saving model checkpoint to results/checkpoint-1000
Configuration saved in results/checkpoint-1000/config.json
Model weights saved in results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in

{'loss': 0.1372, 'learning_rate': 0.0, 'epoch': 1.0}


                                                   
 50%|█████     | 1252/2500 [01:17<21:38,  1.04s/it]

{'eval_loss': 0.26377546787261963, 'eval_precision': 0.8216937097132696, 'eval_recall': 0.8338897410448435, 'eval_f1': 0.8277468036989768, 'eval_accuracy': 0.9230278368630352, 'eval_runtime': 9.9459, 'eval_samples_per_second': 1005.442, 'eval_steps_per_second': 62.84, 'epoch': 1.0}


 60%|█████▉    | 1498/2500 [01:28<00:40, 24.75it/s]Saving model checkpoint to results/checkpoint-1500
Configuration saved in results/checkpoint-1500/config.json
Model weights saved in results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in results/checkpoint-1500/special_tokens_map.json
 80%|███████▉  | 1999/2500 [01:57<00:21, 22.99it/s]Saving model checkpoint to results/checkpoint-2000
Configuration saved in results/checkpoint-2000/config.json
Model weights saved in results/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in results/checkpoint-2000/tokenizer_config.json
Special tokens file saved in results/checkpoint-2000/special_tokens_map.json
100%|██████████| 2500/2500 [02:26<00:00, 24.26it/s]Saving model checkpoint to results/checkpoint-2500
Configuration saved in results/checkpoint-2500/config.json


{'loss': 0.1371, 'learning_rate': 0.0, 'epoch': 2.0}


Model weights saved in results/checkpoint-2500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-2500/tokenizer_config.json
Special tokens file saved in results/checkpoint-2500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 16
                                                   
100%|██████████| 2500/2500 [02:43<00:00, 24.26it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 2500/2500 [02:43<00:00, 15.33it/s]

{'eval_loss': 0.26377546787261963, 'eval_precision': 0.8216937097132696, 'eval_recall': 0.8338897410448435, 'eval_f1': 0.8277468036989768, 'eval_accuracy': 0.9230278368630352, 'eval_runtime': 9.8321, 'eval_samples_per_second': 1017.079, 'eval_steps_per_second': 63.567, 'epoch': 2.0}
{'train_runtime': 163.0462, 'train_samples_per_second': 245.329, 'train_steps_per_second': 15.333, 'train_loss': 0.13715767822265626, 'epoch': 2.0}


TrainOutput(global_step=2500, training_loss=0.13715767822265626, metrics={'train_runtime': 163.0462, 'train_samples_per_second': 245.329, 'train_steps_per_second': 15.333, 'train_loss': 0.13715767822265626, 'epoch': 2.0})

Depois que nosso modelo é treinado, podemos calcular as métricas (`precision`/`recall`/`f1` calculado para cada categoria) no resultado do conjunto `test` do método `predict`

In [28]:
predictions, labels, _ = trainer.predict(tokenized_dataset["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
[label_names[p] for (p, l) in zip(prediction, label) if l != -100]
for prediction, label in zip(predictions, labels)
]

true_labels = [
[label_names[l] for (p, l) in zip(prediction, label) if l != -100]
for prediction, label in zip(predictions, labels)
]


results = metric.compute(predictions=true_predictions, references=true_labels)
results

***** Running Prediction *****
  Num examples = 10000
  Batch size = 16
100%|██████████| 625/625 [00:10<00:00, 61.97it/s]


{'LOC': {'precision': 0.8466921815199563,
  'recall': 0.8853190029727875,
  'f1': 0.8655748700463921,
  'number': 8746},
 'ORG': {'precision': 0.7390180878552972,
  'recall': 0.7260930888575459,
  'f1': 0.7324985771200911,
  'number': 7090},
 'PER': {'precision': 0.8678599221789883,
  'recall': 0.8920172772356423,
  'f1': 0.8797727989902178,
  'number': 6251},
 'overall_precision': 0.8194444444444444,
 'overall_recall': 0.8361026848372346,
 'overall_f1': 0.8276897564036483,
 'overall_accuracy': 0.9242309468822171}

<font color="orange">Observações:</font>

* A `f1 score` para `LOC` e `PER` é `>85%` e `ORG` tem `<75%`

* No geral `f1 score` é de `~83%`

* Podemos melhorar a `accuracy` treinando o modelo `BERT` para um **número maior de epochs** ou ajustando outros parâmetros, como taxa de aprendizado (`Learning rate`), `batch size`, etc.


<font color="yellow">Você pode melhorar o desempenho no conjunto de dados ou experimentar um conjunto de dados de `idioma diferente`.</font>

<font color="orange">Lembrar:</font>

* O modelo `BERT` atinge accuracy de ponta (`state-of-the-art`) em várias tarefas em comparação com outras arquiteturas de RNN. No entanto, `eles exigem alto poder computacional` e levam muito tempo para treinar um modelo, portanto, o ajuste fino (`fine-tune`) é a melhor abordagem para superar os desafios

* A biblioteca `HuggingFace Transformer` facilita o ajuste fino de qualquer tarefa de processamento de linguagem natural (`NLP`) de alto nível, e podemos até ajustar os modelos `pré-treinados` nos conjuntos de dados `personalizados` usando as etapas de pré-processamento necessárias e escolhendo os modelos necessários para a tarefa da biblioteca

* Depois de treinar nosso modelo, é ainda mais fácil compartilhá-lo com a comunidade, carregando-o no `HuggingFace Hub`. Confira o [link a seguir](https://huggingface.co/docs/hub/models-uploading) para saber mais sobre isso.